<a href="https://colab.research.google.com/github/vilcagamarracf/Inv_arroz_METRIC/blob/main/jupyter_notebook/Inv_Arroz_METRIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<h1>Aplicación del modelo METRIC en cultivos de Arroz, Lambayeque</h1>
<h2> Investigación 2021 </h2>
</center>

**Information about this project**
```
FILE          : Inv-Arroz-METRIC.ipynb
AUTHOR        : Cesar Francisco Vilca Gamarra 
EMAIL         : vilcagamarracf@gmail.com 
ORGANIZATION  : UNALM 
CREATION DATE : 29 April, 2021 
DEPENDENCIES  : earthengine-api, os, folium, pandas, matplotlib, numpy ...  
PURPOSE       : Download Landsat Products and have some fun
```
<center>
<img src= 'https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fimage.slideserve.com%2F327001%2Fhow-metric-works6-l.jpg&f=1&nofb=1' width=50%/>
</center>

# *METRIC con Earth Engine*

## 1. Inicio

In [1]:
# Credenciales
import ee
ee.Initialize()
ee.Authenticate()
print('Ya estás logeado.')
print('GEE Version:',ee.__version__)

Enter verification code: 4/1AY0e-g6X8_QCzPXAAeQLyMY67N-QLugebye_oXWnyyZ74kqYr-q9Tr8vh1s

Successfully saved authorization token.
Ya estás logeado.
GEE Version: 0.1.270


Revisar alguna cosita extra que puede eliminarse de la parte 'importar librerías'

In [10]:
# @markdown Importar librerías, funciones y más:
# @markdown - `mapdisplay`: Variable global: `mapViz`
# @markdown - `ee_get_dates`
# @markdown - `conv_fecha`
# @markdown - `ver_rangos`
# @markdown - `ROI CHINCHA`
# @markdown - `create_dateranges`

# mapdisplay
import folium 
def mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    Crea mapas interactivos usando la librería `folium`

    - `center`: Center of the map (Latitude and Longitude) (`list`)
    - `dicc`: Earth Engine Geometries or Tiles dictionary (`dict`)
    - `Tiles=`: 
    Mapbox Bright 
    Mapbox Control Room
    Stamen Terrain
    Stamen Toner
    stamenwatercolor
    cartodbpositron
    - `zoom_start=`: Initial zoom level for the map (`int`)

    Return: A folium.Map object
    '''
    # Centrado de imagen (El [::-1] lee inversamente la lista)
    center = center[::-1]

    # Agregando parámetros de visualización
    global mapViz
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start, control_scale=True)

    # Seteando la visualización de Images
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(tiles = v["tile_fetcher"].url_format,
                         attr  = 'Google Earth Engine',
                         overlay =True,
                         name  = k).add_to(mapViz)
      else:
        folium.GeoJson(data = v,
                       name = k).add_to(mapViz)
    # Agregar: Minimapa
    # from folium.plugins import MiniMap
    # minimap = MiniMap(
    #     tile_layer="OpensTreetMap", 
    #     position="bottomright", 
    #     zoom_level_fixed=4)
    # mapViz.add_child(minimap) # https://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/MiniMap.ipynb

#     Fullscreen (no funciona en colab)
    from folium import plugins
    plugins.Fullscreen().add_to(mapViz)

    # Agregando el control de layers
    mapViz.add_child(folium.LayerControl())
    return mapViz
print('mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10) listo!')

# ee_get_dates ----------------------------------------
from datetime import datetime as dt
def ee_get_dates(ic):
  """
  Permite obtener información de la Fecha y Hora de toma de imágen por el satélite.
  - ic: `ee.ImageCollection()`
  """
  dates = ic.aggregate_array('system:time_start').getInfo()
  anonym = lambda x: dt.utcfromtimestamp(x/1000).strftime('%Y-%m-%d %H:%M:%S')
  return list(map(anonym, dates))
print('ee_get_dates(ic) listo!')

# conv_fecha ----------------------------------------
def conv_fecha(img, getInfo=None):
  '''
  Convierte el formato de UNIX Time al formato %Y-%m-%d %H:%M:%S
  img: ee.Image
  getInfo = True : cuando se tiene ya el .getInfo() 
  '''
  from datetime import datetime as dt
  if getInfo is None:
    return dt.utcfromtimestamp(img.getInfo()['properties']["system:time_start"]/1000).strftime('%Y-%m-%d %H:%M:%S')
  if getInfo==True:
    return dt.utcfromtimestamp(img['properties']["system:time_start"]/1000).strftime('%Y-%m-%d %H:%M:%S')
print('conv_fecha(img, getInfo=None) listo!')

# ROI CHINCHA ----------------------------------------

area_total = ee.FeatureCollection('users/CesarVilca/area_total')
roi_clip = area_total.geometry().bounds()

centroide = [-76.0908, -13.4589]
print('roi, roi_clip y centroide listos!')

# ver_rangos

Start_date = '2012-01-01'
End_date   = '2021-01-31'
range_date = [Start_date, End_date]

def ver_rangos(img_col, range_date= [Start_date, End_date]):
  """
  Devuelve los rangos de fechas existente en un rango inicial escrito manualmente como input.
  Pasos:
  1. `Reducer.minMax()` devuelve un valor min y max
  2. `icol.reduceColumns()` devuelve un diccionario con el min-max
  more info: https://developers.google.com/earth-engine/apidocs/ee-imagecollection-reducecolumns?hl=en
  """
  
  rango = img_col.reduceColumns(ee.Reducer.minMax(), ["system:time_start"]) # Retorna un ee.dictionary: {'max': 1608132402761, 'min': 1545060401000}

  # Obtención de la fecha min y max del image collection en formato ISO standard 8601
  # Javascript trabaja las fechas con milisegundos (se deja así)
  # Python     trabaja las fechas con segundos (por eso /1000)
  init_date = ee.Date(rango.get('min')).getInfo()['value']/1000.
  last_date = ee.Date(rango.get('max')).getInfo()['value']/1000.

  # Dar formato a las fechas  
  from datetime import datetime as dt
  init_date_f = dt.utcfromtimestamp(init_date).strftime('%Y-%m-%d') # %H:%M:%S
  last_date_f = dt.utcfromtimestamp(last_date).strftime('%Y-%m-%d') # %H:%M:%S
  
  # Obtenga el rango de fechas de las imágenes en la colección.
  range_date = [Start_date, End_date]
  print('Total imágenes: {}'.format(img_col.size().getInfo()))
  print('Rango Temporal Ingresado : {} - {}'.format(range_date[0], range_date[1]))
  print('Rango Temporal Real      : {} - {}'.format(init_date_f, last_date_f))
print('ver_rangos(icol, range_date= [Start_date, End_date]) listo!')

# create_dateranges ----------------------------------------
def create_dateranges(init_date, time, range_1, range_2):
  '''
  init_date: '20XX' for monthly dates, '20XX-XX' for daily dates(string)
  time: 'year', 'month', 'day' (string)
  range_1: 1, ... 12 for monthly values (int)
  range_2: 1, ... 13 for monthly values (int)

  Returns a date ranges in a list form.
  '''
  ranges = []
  def create_dates(date, plus_time):
     ''' 
     init_date: string values only, example:  '2013-1'
     time: 'year', 'month', 'day'
  
     more info: https://developers.google.com/earth-engine/apidocs/ee-date-getrange
     '''
     var_time = ee.Date(date).getRange(plus_time).getInfo().get('dates')
  
     from datetime import datetime as dt #-%d
     Start_Date = dt.utcfromtimestamp(var_time[0]/1000).strftime('%Y-%m')
     End_Date   = dt.utcfromtimestamp(var_time[1]/1000).strftime('%Y-%m')
     return  [Start_Date, End_Date, i-1]
  
  for i in range(range_1, range_2+1):
    ranges.append(create_dates(init_date+'-'+str(i), time))
  return ranges
print('create_dateranges(init_date, time, range_1, range_2) listo!')

mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10) listo!
ee_get_dates(ic) listo!
conv_fecha(img, getInfo=None) listo!
roi, roi_clip y centroide listos!
ver_rangos(icol, range_date= [Start_date, End_date]) listo!
create_dateranges(init_date, time, range_1, range_2) listo!


In [3]:
# @markdown *Shapefile del Departamento de Lambayeque*

# 24 Departamentos del Perú
shpPeru = ee.FeatureCollection('users/CesarVilca/Departamentos_Peru')

# Departamento de Lambayeque
shpDep = ee.FeatureCollection('users/CesarVilca/Departamentos_Peru').filter(
    ee.Filter.eq('NOMBDEP', 'LAMBAYEQUE'))

roiPeru = shpPeru.geometry()
roiDep  = shpDep.geometry()

# Centroides
centroidPeru = roiPeru.centroid().coordinates().getInfo()   
centroidDep = roiDep.centroid().coordinates().getInfo()

# https://htmlcolors.com/ para modificar el color
# Visualización de FeatureCollection con .draw
roiPeruDraw = shpPeru.draw(color = "000000", strokeWidth = 2, pointRadius = 2) 
roiDepDraw  = shpDep.draw(color = "00FF00", strokeWidth = 2, pointRadius = 2) 

# dicc_viz1 = {
#     'Perú': roiPeruDraw.getMapId(),
#     'Lambayeque': roiDepDraw.getMapId(),
#     }  
    
# mapdisplay(centroidDep, dicc_viz1, zoom_start=9)

```
Variables
shpPeru  : FeatureCollection Perú
shpDep   : FeatureCollection Lambayeque

roiPeru, roiDep              # geometry
centroidPeru, centroidRoi    # centroides

Coordenadas:
- Chongoyape: [-79.389, -6.639]
- Oyotún    : [-79.309, -6.846]
```

## 2. Selección de imágenes

### Exploración de dataset "Landsat 8 Surface Reflectance"

In [4]:
# @markdown **Prueba**: Visualización rápida de imágenes Landsat y ROI
Start_date = '2020-01-01'  #@param {type: "date"}
End_date   = '2021-01-01'  #@param {type: "date"}
range_date = [Start_date, End_date]

img_col = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")\
.filterDate(Start_date, End_date)\
.filterBounds(roiDep)\
.sort('CLOUD_COVER')\
.median().multiply(0.0001).select(['B4','B3','B2'])

# Visualizacion: Mapdisplay
chongoyape = ee.Geometry.Point([-79.389, -6.639])
oyotun     = ee.Geometry.Point([-79.309, -6.846])
centroide  = ee.Geometry.Point([-79.823, -6.351])

dict_img = {
    'img_col': img_col.getMapId({'min':0.0, 'max':0.3, 'bands': ['B4','B3','B2']}),
    'Lambayeque': roiDepDraw.getMapId(),
    'Chongoyape': chongoyape.getInfo(),
    'centroide': centroide.getInfo(),
    'Oyotun': oyotun.getInfo()
    } # ,'roi': roi_clip.getInfo()

mapdisplay(centroidDep, dict_img, zoom_start=8)

In [5]:
#@markdown Primera imagen registrada: `2013-03-24`

Start_date = '2011-12-17'  #@param {type: "date"}
End_date   = '2021-02-11'  #@param {type: "date"}
range_date = [Start_date, End_date]

img_col1 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")\
        .filterDate(Start_date, End_date)\
        .filterBounds(ee.Geometry.Point(-79.823, -6.351))\
        .sort('CLOUD_COVER')#\.first()

img_col2 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")\
        .filterDate(Start_date, End_date)\
        .filterBounds(ee.Geometry.Point(-79.309, -6.846))\
        .sort('CLOUD_COVER')#\.first()

# .filterBounds(roiDep)\
# .filterBounds(ee.Geometry.Point(-79.82, -6.35))\

bands = ['B4','B3','B2']
img_rgb1 = img_col1.first().multiply(0.0001).select(bands)   #.clip(roi_clip)
img_rgb2 = img_col2.first().multiply(0.0001).select(bands)   #.clip(roi_clip)

# --- Visualizacion de imagen ----------

print('Imagen Top')
ver_rangos(img_col1, range_date=range_date)

print('\nMejor imagen:')
print('% nubosidad:', img_col1.first().getInfo()['properties']['CLOUD_COVER'])
print('Image ID:',img_col1.first().getInfo().get('id'))
print('Fecha:', conv_fecha(img_col1.first()))

print('\nImagen Bottom')
ver_rangos(img_col2, range_date=range_date)

print('\nMejor imagen:')
print('% nubosidad:', img_col2.first().getInfo()['properties']['CLOUD_COVER'])
print('Image ID:',img_col2.first().getInfo().get('id'))
print('Fecha:', conv_fecha(img_col2.first()))

Imagen Top
Total imágenes: 155
Rango Temporal Ingresado : 2011-12-17 - 2021-02-11
Rango Temporal Real      : 2013-03-22 - 2021-02-03

Mejor imagen:
% nubosidad: 5.4
Image ID: LANDSAT/LC08/C01/T1_SR/LC08_010064_20200811
Fecha: 2020-08-11 15:27:52

Imagen Bottom
Total imágenes: 149
Rango Temporal Ingresado : 2011-12-17 - 2021-02-11
Rango Temporal Real      : 2013-03-27 - 2021-02-03

Mejor imagen:
% nubosidad: 0.75
Image ID: LANDSAT/LC08/C01/T1_SR/LC08_010065_20160816
Fecha: 2016-08-16 15:28:22


In [11]:
#@markdown Visualización
chongoyape = ee.Geometry.Point([-79.389, -6.639])
oyotun     = ee.Geometry.Point([-79.309, -6.846])
centroide  = ee.Geometry.Point([-79.823, -6.351])

dict_img = {
    'imgtop': img_rgb1.clip(roiDep).getMapId({'min':0.0, 'max':0.3, 'bands': bands}),
    'imgbot': img_rgb2.clip(roiDep).getMapId({'min':0.0, 'max':0.3, 'bands': bands}),

    # 'Lambayeque': roiDepDraw.getMapId(),
    'Chongoyape': chongoyape.getInfo(),

    'centroide': centroide.getInfo(),
    'Oyotun': oyotun.getInfo()
    }

mapdisplay(centroidDep, dict_img, zoom_start=9)

### Generar visualización a partir de IDs

Para visualizar rápidamente imágenes landsat que abarquen un territorio usar:
- [GEO GPS PERÚ: Ubica el PATH y ROW de tu distrito para imágenes LANDSAT](https://www.geogpsperu.com/2016/02/ubica-el-path-y-row-de-tu-distrito-para.html)

In [ ]:
# @markdown Generador de IDs Anual

longitud = -79.809 #@param [] {type:"raw", allow-input: true}
latitud = -6.746 #@param [] {type:"raw", allow-input: true}
coordenada = list([longitud,latitud])

periodo = create_dateranges('2020','month', 1, 12)

ID_snippet_name = "COPERNICUS/S2_SR" #@param ["LANDSAT/LE07/C01/T1_SR", "LANDSAT/LC08/C01/T1", "LANDSAT/LC08/C01/T1_TOA", "LANDSAT/LC08/C01/T1_SR", "COPERNICUS/S2_SR"]

imgCol_generado = [ee.ImageCollection(ID_snippet_name)\
             .filterDate(periodo[i][0], periodo[i][1])\
             .filterBounds(ee.Geometry.Point(coordenada)) for i in range(12)] 

# Cantidad de imágenes por mes 
imgCol_generado_size = [imgCol_generado[i].size().getInfo() for i in range(12)]

# Diccionario de Meses con su imágenes correspondientes
imgCol_dict = {}

for i in range(12):
  IDs = []
  imgCol_dict[i+1] = IDs
  for j in range(imgCol_generado_size[i]):
    IDs.append(imgCol_generado[i].getInfo()['features'][j]['id']) 

# imgCol_dict

#### Diccionarios generados

Para darle buen formato a los diccionarios, usar:
- [Python Formatter](https://codebeautify.org/python-formatter-beautifier)

In [ ]:
# @markdown Diccionario Sentinel 2: `imgCol_dict`
imgCol_dict = {
    1: [
        "COPERNICUS/S2_SR/20200102T153611_20200102T153743_T17MPN",
        "COPERNICUS/S2_SR/20200107T153619_20200107T154047_T17MPN",
        "COPERNICUS/S2_SR/20200112T153611_20200112T153613_T17MPN",
        "COPERNICUS/S2_SR/20200117T153619_20200117T153613_T17MPN",
        "COPERNICUS/S2_SR/20200122T153611_20200122T153612_T17MPN",
        "COPERNICUS/S2_SR/20200127T153609_20200127T153611_T17MPN",
    ],
    2: [
        "COPERNICUS/S2_SR/20200201T153611_20200201T153611_T17MPN",
        "COPERNICUS/S2_SR/20200206T153619_20200206T153613_T17MPN",
        "COPERNICUS/S2_SR/20200211T153611_20200211T153626_T17MPN",
        "COPERNICUS/S2_SR/20200216T153619_20200216T154305_T17MPN",
        "COPERNICUS/S2_SR/20200221T153611_20200221T154337_T17MPN",
        "COPERNICUS/S2_SR/20200226T153619_20200226T154119_T17MPN",
    ],
    3: [
        "COPERNICUS/S2_SR/20200302T153611_20200302T153614_T17MPN",
        "COPERNICUS/S2_SR/20200307T153619_20200307T153616_T17MPN",
        "COPERNICUS/S2_SR/20200312T153621_20200312T153615_T17MPN",
        "COPERNICUS/S2_SR/20200317T153619_20200317T154424_T17MPN",
        "COPERNICUS/S2_SR/20200322T153621_20200322T154411_T17MPN",
        "COPERNICUS/S2_SR/20200327T153619_20200327T154304_T17MPN",
    ],
    4: [
        "COPERNICUS/S2_SR/20200401T153621_20200401T153901_T17MPN",
        "COPERNICUS/S2_SR/20200406T153619_20200406T154234_T17MPN",
        "COPERNICUS/S2_SR/20200411T153621_20200411T153910_T17MPN",
        "COPERNICUS/S2_SR/20200416T153619_20200416T154403_T17MPN",
        "COPERNICUS/S2_SR/20200421T153621_20200421T154337_T17MPN",
        "COPERNICUS/S2_SR/20200426T153619_20200426T153904_T17MPN",
    ],
    5: [
        "COPERNICUS/S2_SR/20200501T153621_20200501T153850_T17MPN",
        "COPERNICUS/S2_SR/20200506T153619_20200506T153616_T17MPN",
        "COPERNICUS/S2_SR/20200511T153621_20200511T154343_T17MPN",
        "COPERNICUS/S2_SR/20200516T153619_20200516T153618_T17MPN",
        "COPERNICUS/S2_SR/20200516T153619_20200516T154501_T17MPN",
        "COPERNICUS/S2_SR/20200521T153631_20200521T154352_T17MPN",
        "COPERNICUS/S2_SR/20200526T153619_20200526T154127_T17MPN",
        "COPERNICUS/S2_SR/20200531T153631_20200531T153940_T17MPN",
    ],
    6: [
        "COPERNICUS/S2_SR/20200605T153619_20200605T153834_T17MPN",
        "COPERNICUS/S2_SR/20200610T153631_20200610T154258_T17MPN",
        "COPERNICUS/S2_SR/20200615T153619_20200615T153722_T17MPN",
        "COPERNICUS/S2_SR/20200620T153631_20200620T154446_T17MPN",
        "COPERNICUS/S2_SR/20200625T153619_20200625T153621_T17MPN",
        "COPERNICUS/S2_SR/20200625T153619_20200625T154504_T17MPN",
        "COPERNICUS/S2_SR/20200630T153621_20200630T153946_T17MPN",
    ],
    7: [
        "COPERNICUS/S2_SR/20200705T153619_20200705T153621_T17MPN",
        "COPERNICUS/S2_SR/20200710T153621_20200710T154423_T17MPN",
        "COPERNICUS/S2_SR/20200715T153619_20200715T153620_T17MPN",
        "COPERNICUS/S2_SR/20200715T153619_20200715T154503_T17MPN",
        "COPERNICUS/S2_SR/20200720T153621_20200720T154406_T17MPN",
        "COPERNICUS/S2_SR/20200725T153619_20200725T153621_T17MPN",
        "COPERNICUS/S2_SR/20200730T153631_20200730T154345_T17MPN",
    ],
    8: [
        "COPERNICUS/S2_SR/20200804T153619_20200804T154255_T17MPN",
        "COPERNICUS/S2_SR/20200809T153631_20200809T154345_T17MPN",
        "COPERNICUS/S2_SR/20200814T153619_20200814T153738_T17MPN",
        "COPERNICUS/S2_SR/20200819T153631_20200819T153625_T17MPN",
        "COPERNICUS/S2_SR/20200824T153619_20200824T154230_T17MPN",
        "COPERNICUS/S2_SR/20200829T153621_20200829T154210_T17MPN",
    ],
    9: [
        "COPERNICUS/S2_SR/20200903T153619_20200903T154357_T17MPN",
        "COPERNICUS/S2_SR/20200908T153621_20200908T154046_T17MPN",
        "COPERNICUS/S2_SR/20200913T153619_20200913T153628_T17MPN",
        "COPERNICUS/S2_SR/20200918T153621_20200918T153624_T17MPN",
        "COPERNICUS/S2_SR/20200923T153619_20200923T153831_T17MPN",
        "COPERNICUS/S2_SR/20200928T153621_20200928T153624_T17MPN",
    ],
    10: [
        "COPERNICUS/S2_SR/20201003T153619_20201003T153759_T17MPN",
        "COPERNICUS/S2_SR/20201008T153631_20201008T153625_T17MPN",
        "COPERNICUS/S2_SR/20201013T153619_20201013T153807_T17MPN",
        "COPERNICUS/S2_SR/20201018T153621_20201018T153841_T17MPN",
        "COPERNICUS/S2_SR/20201023T153619_20201023T154139_T17MPN",
        "COPERNICUS/S2_SR/20201028T153621_20201028T154210_T17MPN",
    ],
    11: [
        "COPERNICUS/S2_SR/20201102T153619_20201102T154334_T17MPN",
        "COPERNICUS/S2_SR/20201107T153621_20201107T154039_T17MPN",
        "COPERNICUS/S2_SR/20201112T153619_20201112T154104_T17MPN",
        "COPERNICUS/S2_SR/20201117T153621_20201117T154056_T17MPN",
        "COPERNICUS/S2_SR/20201122T153619_20201122T154415_T17MPN",
        "COPERNICUS/S2_SR/20201127T153621_20201127T154054_T17MPN",
    ],
    12: [
        "COPERNICUS/S2_SR/20201202T153619_20201202T154414_T17MPN",
        "COPERNICUS/S2_SR/20201207T153621_20201207T154054_T17MPN",
        "COPERNICUS/S2_SR/20201212T153619_20201212T154111_T17MPN",
        "COPERNICUS/S2_SR/20201222T153619_20201222T154433_T17MPN",
        "COPERNICUS/S2_SR/20201227T153621_20201227T154110_T17MPN",
    ],
}

In [ ]:
# @markdown Diccionario Landsat 8 TOA: `imgCol_dict`

imgCol_dict = {
    1: ['LANDSAT/LC08/C01/T1_TOA/LC08_010065_20200116'],
    2: ['LANDSAT/LC08/C01/T1_TOA/LC08_010065_20200201',
      'LANDSAT/LC08/C01/T1_TOA/LC08_010065_20200217'],
    3: ['LANDSAT/LC08/C01/T1_TOA/LC08_010065_20200304',
      'LANDSAT/LC08/C01/T1_TOA/LC08_010065_20200320'],
    4: ['LANDSAT/LC08/C01/T1_TOA/LC08_010065_20200405',
      'LANDSAT/LC08/C01/T1_TOA/LC08_010065_20200421'],
    5: ['LANDSAT/LC08/C01/T1_TOA/LC08_010065_20200507',
      'LANDSAT/LC08/C01/T1_TOA/LC08_010065_20200523'],
    6: ['LANDSAT/LC08/C01/T1_TOA/LC08_010065_20200608',
      'LANDSAT/LC08/C01/T1_TOA/LC08_010065_20200624'],
    7: ['LANDSAT/LC08/C01/T1_TOA/LC08_010065_20200726'],
    8: ['LANDSAT/LC08/C01/T1_TOA/LC08_010065_20200811',
      'LANDSAT/LC08/C01/T1_TOA/LC08_010065_20200827'],
    9: ['LANDSAT/LC08/C01/T1_TOA/LC08_010065_20200912',
      'LANDSAT/LC08/C01/T1_TOA/LC08_010065_20200928'],
    10: ['LANDSAT/LC08/C01/T1_TOA/LC08_010065_20201014',
      'LANDSAT/LC08/C01/T1_TOA/LC08_010065_20201030'],
    11: ['LANDSAT/LC08/C01/T1_TOA/LC08_010065_20201115'],
    12: ['LANDSAT/LC08/C01/T1_TOA/LC08_010065_20201201']
 }

# @markdown Diccionario Landsat 8 Raw Scenes: `imgCol_dict`

imgCol_dict = {
    1: ['LANDSAT/LC08/C01/T1/LC08_010065_20200116'],
    2: ['LANDSAT/LC08/C01/T1/LC08_010065_20200201',
      'LANDSAT/LC08/C01/T1/LC08_010065_20200217'],
    3: ['LANDSAT/LC08/C01/T1/LC08_010065_20200304',
      'LANDSAT/LC08/C01/T1/LC08_010065_20200320'],
    4: ['LANDSAT/LC08/C01/T1/LC08_010065_20200405',
      'LANDSAT/LC08/C01/T1/LC08_010065_20200421'],
    5: ['LANDSAT/LC08/C01/T1/LC08_010065_20200507',
      'LANDSAT/LC08/C01/T1/LC08_010065_20200523'],
    6: ['LANDSAT/LC08/C01/T1/LC08_010065_20200608',
      'LANDSAT/LC08/C01/T1/LC08_010065_20200624'],
    7: ['LANDSAT/LC08/C01/T1/LC08_010065_20200726'],
    8: ['LANDSAT/LC08/C01/T1/LC08_010065_20200811',
      'LANDSAT/LC08/C01/T1/LC08_010065_20200827'],
    9: ['LANDSAT/LC08/C01/T1/LC08_010065_20200912',
      'LANDSAT/LC08/C01/T1/LC08_010065_20200928'],
    10: ['LANDSAT/LC08/C01/T1/LC08_010065_20201014',
      'LANDSAT/LC08/C01/T1/LC08_010065_20201030'],
    11: ['LANDSAT/LC08/C01/T1/LC08_010065_20201115'],
    12: ['LANDSAT/LC08/C01/T1/LC08_010065_20201201']
}

In [ ]:
# @markdown Diccionario Landsat 7: `imgCol_dict_L7`

imgCol_dict = {
    1: ['LANDSAT/LE07/C01/T1_SR/LE07_010065_20200108',
        'LANDSAT/LE07/C01/T1_SR/LE07_010065_20200124'],
    2: ['LANDSAT/LE07/C01/T1_SR/LE07_010065_20200209',
      'LANDSAT/LE07/C01/T1_SR/LE07_010065_20200225'],
    3: ['LANDSAT/LE07/C01/T1_SR/LE07_010065_20200312',
      'LANDSAT/LE07/C01/T1_SR/LE07_010065_20200328'],
    4: ['LANDSAT/LE07/C01/T1_SR/LE07_010065_20200429'],
    5: ['LANDSAT/LE07/C01/T1_SR/LE07_010065_20200515',
      'LANDSAT/LE07/C01/T1_SR/LE07_010065_20200531'],
    6: [],
    7: ['LANDSAT/LE07/C01/T1_SR/LE07_010065_20200702',
      'LANDSAT/LE07/C01/T1_SR/LE07_010065_20200718'],
    8: ['LANDSAT/LE07/C01/T1_SR/LE07_010065_20200803',
      'LANDSAT/LE07/C01/T1_SR/LE07_010065_20200819'],
    9: ['LANDSAT/LE07/C01/T1_SR/LE07_010065_20200904',
      'LANDSAT/LE07/C01/T1_SR/LE07_010065_20200920'],
    10: ['LANDSAT/LE07/C01/T1_SR/LE07_010065_20201006',
      'LANDSAT/LE07/C01/T1_SR/LE07_010065_20201022'],
    11: ['LANDSAT/LE07/C01/T1_SR/LE07_010065_20201107',
      'LANDSAT/LE07/C01/T1_SR/LE07_010065_20201123'],
    12: ['LANDSAT/LE07/C01/T1_SR/LE07_010065_20201209',
      'LANDSAT/LE07/C01/T1_SR/LE07_010065_20201225']
  }

#### Visualización de imágenes Landsat por meses (2020)

In [ ]:
# @markdown Diccionario Landsat 8 SR: `imgCol_dict`

imgCol_dict = {
    1: ['LANDSAT/LC08/C01/T1_SR/LC08_010065_20200116'],
    2: [
        'LANDSAT/LC08/C01/T1_SR/LC08_010065_20200201',
        'LANDSAT/LC08/C01/T1_SR/LC08_010065_20200217'
    ],  # buena foto
    3: [
        'LANDSAT/LC08/C01/T1_SR/LC08_010065_20200304',
        'LANDSAT/LC08/C01/T1_SR/LC08_010065_20200320'
    ],
    4: [
        'LANDSAT/LC08/C01/T1_SR/LC08_010065_20200405',
        'LANDSAT/LC08/C01/T1_SR/LC08_010065_20200421'
    ],  # buena foto
    5: [
        'LANDSAT/LC08/C01/T1_SR/LC08_010065_20200507',
        'LANDSAT/LC08/C01/T1_SR/LC08_010065_20200523'
    ],
    6: [
        'LANDSAT/LC08/C01/T1_SR/LC08_010065_20200608',
        'LANDSAT/LC08/C01/T1_SR/LC08_010065_20200624'
    ],
    7: ['LANDSAT/LC08/C01/T1_SR/LC08_010065_20200726'],
    8: [
        'LANDSAT/LC08/C01/T1_SR/LC08_010065_20200811',
        'LANDSAT/LC08/C01/T1_SR/LC08_010065_20200827'
    ],
    9: [
        'LANDSAT/LC08/C01/T1_SR/LC08_010065_20200912',
        'LANDSAT/LC08/C01/T1_SR/LC08_010065_20200928'
    ],
    10: [
        'LANDSAT/LC08/C01/T1_SR/LC08_010065_20201014',
        'LANDSAT/LC08/C01/T1_SR/LC08_010065_20201030'
    ],
    11: ['LANDSAT/LC08/C01/T1_SR/LC08_010065_20201115'],
    12: ['LANDSAT/LC08/C01/T1_SR/LC08_010065_20201201']
}

In [ ]:
# @markdown Visualización interactiva de imágenes con la librería `interact`

from ipywidgets import interact 
# https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html#
from pprint import pprint

Natural_Color  = ['B4', 'B3', 'B2']

def visualizar_mes(mes):

  # Diccionario para visualización
  viz_dict = {}

  # Generar diccionario de visualización
  for i in range(len(imgCol_dict[mes])):
    viz_dict[
      conv_fecha(ee.Image(imgCol_dict[mes][i])) 
      ] \
      = ee.Image(imgCol_dict[mes][i])\
      .divide(10000)\
      .clip(roiDep)\
      .getMapId({'min':0,'max':0.3,'bands':Natural_Color})

  # Presentar ID's
  print("\nID's disponibles:")
  pprint(imgCol_dict[mes])
  print('')

  # Mostrar el mapa leaflet
  return mapdisplay([-79.809, -6.746], viz_dict, zoom_start=10)

interact(visualizar_mes, mes=[1,2,3,4,5,6,7,8,9,10,11,12]);

interactive(children=(Dropdown(description='mes', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12), value=1), O…

### Generar tabla de resultados

In [ ]:
latitud = -6.846 
longitud = -79.309 
coordenada = list([longitud,latitud])

# icol_raw = ee.ImageCollection("LANDSAT/LC08/C01/T1").filterDate('2020-1-1','2021-2-1').filterBounds(ee.Geometry.Point(coordenada))
# print(icol_raw.size().getInfo())
# icol_toa = ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA").filterDate('2020-1-1','2021-2-1').filterBounds(ee.Geometry.Point(coordenada))
# print(icol_toa.size().getInfo())
icol_sr = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR").filterDate('2020-1-1','2021-2-1').filterBounds(ee.Geometry.Point(coordenada))
print(icol_sr.size().getInfo())

lista_fechas = icol_sr.aggregate_array('system:time_start').getInfo()
cantidad = len(lista_fechas)
# ee_get_dates(icol)

21


In [ ]:
import pandas as pd

df = pd.DataFrame(lista_fechas, columns = ['millis'])

def add_date_info(df):
  df['Número'] = [i for i in range(1, cantidad + 1)]
  df['Timestamp'] = pd.to_datetime(df['millis'], unit='ms')
  df['Día Juliano'] = pd.DatetimeIndex(df['Timestamp']).dayofyear
  df['Sensor'] = ['OLI/TIRS' for i in range(1, cantidad + 1)]
  df['Año'] = pd.DatetimeIndex(df['Timestamp']).year
  df['Mes'] = pd.DatetimeIndex(df['Timestamp']).month
  df['Día'] = pd.DatetimeIndex(df['Timestamp']).day
  
  df = df.drop(columns=['millis'])
  df = df.drop(columns=['Timestamp'])
  return df

df = add_date_info(df)
print(df)

# Exportar como .csv
# df.to_csv('datos_2020_L8.csv')

    Número  Día Juliano    Sensor   Año  Mes  Día
0        1           16  OLI/TIRS  2020    1   16
1        2           32  OLI/TIRS  2020    2    1
2        3           48  OLI/TIRS  2020    2   17
3        4           64  OLI/TIRS  2020    3    4
4        5           80  OLI/TIRS  2020    3   20
5        6           96  OLI/TIRS  2020    4    5
6        7          112  OLI/TIRS  2020    4   21
7        8          128  OLI/TIRS  2020    5    7
8        9          144  OLI/TIRS  2020    5   23
9       10          160  OLI/TIRS  2020    6    8
10      11          176  OLI/TIRS  2020    6   24
11      12          208  OLI/TIRS  2020    7   26
12      13          224  OLI/TIRS  2020    8   11
13      14          240  OLI/TIRS  2020    8   27
14      15          256  OLI/TIRS  2020    9   12
15      16          272  OLI/TIRS  2020    9   28
16      17          288  OLI/TIRS  2020   10   14
17      18          304  OLI/TIRS  2020   10   30
18      19          320  OLI/TIRS  2020   11   15


## 3. Balance de energía (en proceso)

Recursos de $\LaTeX$
- [LaTex mathematics for equations](https://en.wikibooks.org/wiki/LaTeX/Mathematics)
- [Overleaf - Aligning equations with amsmath](https://www.overleaf.com/learn/latex/Aligning_equations_with_amsmath)

Landsat 8 : [info aquí](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_SR)

|Name	|Units|	Scale|	Wavelength|	Description|
|:-:|:-:|-|-|-|
|B1		|-|0.0001	|0.435-0.451 μm	|Band 1 (ultra blue) surface reflectance|
|B2		|-|0.0001	|0.452-0.512 μm	|Band 2 (blue) surface reflectance|
|B3		|-|0.0001	|0.533-0.590 μm	|Band 3 (green) surface reflectance|
|B4		|-|0.0001	|0.636-0.673 μm	|Band 4 (red) surface reflectance|
|B5		|-|0.0001	|0.851-0.879 μm	|Band 5 (near infrared) surface reflectance|
|B6		|-|0.0001	|1.566-1.651 μm	|Band 6 (shortwave infrared 1) surface reflec|-tance|
|B7		|-|0.0001	|2.107-2.294 μm	|Band 7 (shortwave infrared 2) surface reflectance|
|B10	|Kelvin	|0.1	|10.60-11.19 μm|Band 10 brightness temperature.|
|B11	|Kelvin	|0.1	|11.50-12.51 μm|Band 11 brightness temperature.|
|sr_aerosol				|Aerosol attributes

Band B10 and B11, while originally collected with a resolution of 100m / pixel, has been resampled using cubic convolution to 30m.

- [Conversión de valores de ND a radiancia](https://www.youtube.com/watch?v=kRN1ekslBH0&list=PLkMtpdwUewHG5JvNoZbOL7-c6_EXoS_UI)

### Albedo $\alpha_{s}$

- Coeficientes de ponderación de la irradiancia exoatmosférica de la banda $\lambda$

$$
\alpha_s = \sum_{b=1}^{n}[\rho_{s,b} \times \omega_b]
$$

- $\rho_\lambda$ : Valor de la reflectancia de la banda $\lambda$
- $\omega_\lambda$ : Coef. de ponderación de la irradiancia exoatmosférica de la banda $\lambda$ 

In [ ]:
# @markdown - Paleta NDVI (Yuri SIG) 20 Colores: `yuri_ndvi_pal`
yuri_ndvi_pal = [
    'ffffff', 'fffcff', 'fff9ff', 'fff7ff', 'fff4ff', 'fff2ff', 'ffefff',
    'ffecff', 'ffeaff', 'ffe7ff', 'ffe5ff', 'ffe2ff', 'ffe0ff', 'ffddff',
    'ffdaff', 'ffd8ff', 'ffd5ff', 'ffd3ff', 'ffd0ff', 'ffceff', 'ffcbff',
    'ffc8ff', 'ffc6ff', 'ffc3ff', 'ffc1ff', 'ffbeff', 'ffbcff', 'ffb9ff',
    'ffb6ff', 'ffb4ff', 'ffb1ff', 'ffafff', 'ffacff', 'ffaaff', 'ffa7ff',
    'ffa4ff', 'ffa2ff', 'ff9fff', 'ff9dff', 'ff9aff', 'ff97ff', 'ff95ff',
    'ff92ff', 'ff90ff', 'ff8dff', 'ff8bff', 'ff88ff', 'ff85ff', 'ff83ff',
    'ff80ff', 'ff7eff', 'ff7bff', 'ff79ff', 'ff76ff', 'ff73ff', 'ff71ff',
    'ff6eff', 'ff6cff', 'ff69ff', 'ff67ff', 'ff64ff', 'ff61ff', 'ff5fff',
    'ff5cff', 'ff5aff', 'ff57ff', 'ff55ff', 'ff52ff', 'ff4fff', 'ff4dff',
    'ff4aff', 'ff48ff', 'ff45ff', 'ff42ff', 'ff40ff', 'ff3dff', 'ff3bff',
    'ff38ff', 'ff36ff', 'ff33ff', 'ff30ff', 'ff2eff', 'ff2bff', 'ff29ff',
    'ff26ff', 'ff24ff', 'ff21ff', 'ff1eff', 'ff1cff', 'ff19ff', 'ff17ff',
    'ff14ff', 'ff12ff', 'ff0fff', 'ff0cff', 'ff0aff', 'ff07ff', 'ff05ff',
    'ff02ff', 'ff00ff', 'ff00ff', 'ff0af4', 'ff15e9', 'ff1fdf', 'ff2ad4',
    'ff35c9', 'ff3fbf', 'ff4ab4', 'ff55aa', 'ff5f9f', 'ff6a94', 'ff748a',
    'ff7f7f', 'ff8a74', 'ff946a', 'ff9f5f', 'ffaa55', 'ffb44a', 'ffbf3f',
    'ffc935', 'ffd42a', 'ffdf1f', 'ffe915', 'fff40a', 'ffff00', 'ffff00',
    'fffb00', 'fff700', 'fff300', 'fff000', 'ffec00', 'ffe800', 'ffe400',
    'ffe100', 'ffdd00', 'ffd900', 'ffd500', 'ffd200', 'ffce00', 'ffca00',
    'ffc600', 'ffc300', 'ffbf00', 'ffbb00', 'ffb700', 'ffb400', 'ffb000',
    'ffac00', 'ffa800', 'ffa500', 'ffa500', 'f7a400', 'f0a300', 'e8a200',
    'e1a200', 'd9a100', 'd2a000', 'ca9f00', 'c39f00', 'bb9e00', 'b49d00',
    'ac9c00', 'a59c00', '9d9b00', '969a00', '8e9900', '879900', '7f9800',
    '789700', '709700', '699600', '619500', '5a9400', '529400', '4b9300',
    '439200', '349100', '2d9000', '258f00', '1e8e00', '168e00', '0f8d00',
    '078c00', '008c00', '008c00', '008700', '008300', '007f00', '007a00',
    '007600', '007200', '006e00', '006900', '006500', '006100', '005c00',
    '005800', '005400', '005000', '004c00'
]

In [ ]:
image_ID = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_010065_20200116')

In [ ]:
# Albedo 
image = image_ID.multiply(0.0001)

# Coeficientes de ponderación por banda (Da Silva et al., 2016)
# https://www.scielo.br/j/rbeaa/a/sX6cJjNXWMfHQ5p4h33B8Zz/?lang=en

albedo = image.expression(
    '(0.3*B2 + 0.277*B3 + 0.233*B4 + 0.143*B5 + 0.036*B6 + 0.012*B7)',
    {
        'B2' : image.select('B2'),
        'B3' : image.select('B3'),
        'B4' : image.select('B4'),
        'B5' : image.select('B5'),
        'B6' : image.select('B6'),
        'B7' : image.select('B7')
    }
)

palette = [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ]
viz_params = {'min': 0.0, 'max': 0.5, 'palette': palette};

mapdisplay([-79.309, -6.846], {'Albedo':albedo.getMapId(viz_params),
                               'Imagen':image.getMapId({'min':0, 'max':0.3, 'bands':['B4','B3','B2']})},zoom_start=9)

# mapdisplay([-79.309, -6.846], {'Albedo':albedo.clip(roiDep).getMapId(viz_params)},zoom_start=8)

### Radiación de onda corta entrante $R_{S\downarrow}$

Morse et al. (2000)

$$
R_{S\downarrow} = G_{sc} \times \cos(\theta) \times d_r \times \tau_{sw}
$$

Dónde:
- $R_{S\downarrow}$ : Radiación de onda corte entrante $[W/m²]$
- $G_{sc}$ : Constante solar $[1367 \space W/m²]$
- $\theta$ : Ángulo zenital solar $[rad]$
- $d_r$ : Inversa de la distancia Sol-Tierra $[m^{-1}]$
- $\tau_{sw}$ : Es la transmitancia en un sentido con condiciones de claridad $[W/m² \space K]$

$$
\tau_{SW} = \tau_B + \tau_D
$$

Donde:
- $\tau_B$ es el índice de transmisividad de radiación directa y 
- $\tau_D$ es índice de transmisividad de radiación difusa

La ecuación utilizada para el cálcula de $\tau_B$ es:

$$
\tau_B = 0.98 * \exp \left[ \frac{-0.00146*P_{air}}{K_t * \cos\theta} - 0.075 * \left( \frac {W}{\cos\theta}\right)^{0.4} \right]
$$

$$
P_{air} = 101.1 \left( \frac {293 - 0.0067 * z}{293}\right)^{5.26}
$$

In [ ]:
image_ID.getInfo()

In [ ]:
import math

ang_zenital = image_ID.get('SOLAR_ZENITH_ANGLE').getInfo()*math.pi/180
G_sc = 1367
dr = 1/image_ID.get('EARTH_SUN_DISTANCE').getInfo()

1.0166300340977714

In [ ]:
1/0.983642

1.0166300340977714

In [ ]:
# imagen = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_010065_20200116')
zenitAngle = image.get('SOLAR_ZENITH_ANGLE').getInfo()
zenitAngle

In [ ]:
R_s_entrante = 1367*cos(zenitAngle)*d_r*tau_sw

### Radiación de onda larga saliente $R_{L\uparrow}$

- Morse et al., 2000.

$$
R_{L\downarrow} = \epsilon_0 * \sigma * T_{s}^{4}
$$

Dónde:
- $R_{L\downarrow}$ : Radiación de onda larga saliente $[W/m²]$
- $\epsilon_0$ : Emisividad del ancho de banda de la superficie
- $\sigma$ : Constante de Stefan-Boltzmann $(5.67*10^{-8}[W/m²])$
- $T_s$ : Temperatura de brillo de superficie $[K]$

Para hallar la emisividad:

$$
\epsilon_0 = 0.95 + 0.01 * LAI \text{, cuando NDVI > 0 y LAI < 3} 
$$

Dónde:
- $\text{LAI}$ : es el índice de área foliar
- $\text{NDVI}$ : El índice de vegetación de diferencia normalizada

In [ ]:
R_l_saliente = emisividad*(5.67*10**-8)*temp_brillo**4

### Radiación de onda larga entrante $R_{L\downarrow}$

Bastiaansen et al. 1998:

$$
R_{L\downarrow} = 1.08 * (-\ln({\tau_{sw})})^{0.265} * \sigma * T_{0 ref}^{4}
$$

Dónde:
- $R_{L\downarrow}$ : Radiación de onda corta entrante
- $\tau_{sw}$ : Es la transmitancia en un sentido con condiciones de claridad
- $\sigma$ : Constante de Stefan-Boltzmann $(5.67*10^{-8} [W/m^{2}])$
- $T_{0 ref}$ : Temperatura en un punto de referencia con buen riego


### Radiación neta de la superficie $R_n$

Morse et al. 2000:

$$
R_n = (1- \alpha)R_{S\downarrow} + (R_{L\downarrow} - R_{L\uparrow}) - (1- \epsilon_0)R_{L\downarrow}
$$

Dónde: 
- $R_n$ : Flujo de radiación neta $[W/m^2]$
- $\alpha$ : Albedo de superficie
- $R_{S\downarrow}$ : Radiación de onda corta entrante $[W/m^2]$
- $R_{L\downarrow}$ : Radiación de onda larga entrante $[W/m^2]$
- $R_{L\uparrow}$ : Radiación de onda larga saliente $[W/m^2]$
- $\epsilon_0$ : Emisividad del ancho de banda en la superficie

### Flujo de calor del suelo $G$

> **Forma 1**
>
>Choudhury, Idso y Reginato (1987) o Allen, Pereira, Raes y Smith (1998)
>
>$$
G = 0.4 * \mathrm{e}^{-0.5* \text{LAI}} * R_n
$$
>
>Dónde:
>- $G$ : Flujo de calor del suelo $[W/m^{2}]$
>- $\text{LAI}$ : Índice de área foliar
>- $R_n$ : Flujo de radiación neta $[W/m^{2}]$

> **Forma 2**
>
> Despeje de la relación de calor almacenado en el suelo y vegetación debida a la conducción $R_n$, según Bastiaanssen (2000)
>
>$$
\frac{G}{R_n} = \frac{T_s}{\alpha}(0.0038\alpha + 0.0074\alpha^{2})(1-0.98*\text{NDVI}^4) 
$$
> Dónde:
> - $\frac{G}{R_n}$ : Relación del calor almacenado en suelo y vegetación
> - $T_s$ : Temperatura de brillo de superficie $[K]$
> - $\alpha$ : Albedo de superficie
> - $\text{NDVI}$ : Índice de vegetación de diferencia normalizada

### Momento de resistencia aerodinámica (MRA) o $Z_{om}$

Chemin y Din Ahmad (2000), toman en cuenta las condiciones aerodinámicas de la vegetación la cual está en función de su altura y distribución espacial.

$$
Z_{om} = a + b*\text{NDVI}
$$

Dónde:

$$
a = \ln(0.02) - [b*(0.02)]
$$

$$
b = \frac{(\text{NDVI}_{max}-0.02)}{\ln(\frac{h_v}{7}-\ln(0.002)}
$$

- $\text{NDVI}$ : Índice de vegetación de diferencia normalizada
- $\text{NDVI}_{max}$ : Valor local máximo del $\text{NDVI}$
- $h_v$ : altura promedio de la vegetación cuando se tiene $\text{NDVI}_{max}$

### Selección de pixel caliente y frío

Requisito para conocer el aporte de flujo de calor sensible $H$ en el modelo de balance, cada pixel con vegetación requerirá de conocer un gradiente de temperatura $\delta T$ en referencia a su condición aerodinámica a fin de estimar la densidad de su aporte, esto se logra a partir de la relación lineal que guardan las temperaturas entre dos superficies extremas con características distintas. 

Se han propuesto metodologías para la selección de estos puntos extremos denominándose **pixel frío** y **pixel caliente**, donde: 
- un **pixel frío** representa una superficie con vegetación densa y húmeda, mientras 
- un **pixel caliente** representa una superficie seca sin vegetación. 

Según el método de Olmedo et al. (2016), la selección de estos dos parámetros puede llevarse sobre el área de estudio y localizarlos mediante el cumplimiento de una serie de condiciones sobre otros parámetros conocidos como se muestra a continuación:

$\space$

$$
\text{pixel frio} = 
\begin{cases}
  \begin{gather*}
    3 \le \text{LAI} \le 6 \\
    0.18 \le \alpha \le 0.25 \\
    0.03 \le Z_{0m} \le 0.08 \\
    \text{NDVI} \ge \text{NDVI}_{max} - 0.15 \\
    T_s < T_{max} - 5 
  \end{gather*}
\end{cases}
$$

$\space$

$$
\text{pixel caliente} = 
\begin{cases}
  \begin{gather*}
    0.13 \le \alpha \le 0.15 \\
    Z_{0m} \le 0.005 \\
    0.1 \le \text{NDVI} \le 0.28 \\
    T_s > T_{max} - 5
  \end{gather*}
\end{cases}
$$

Dónde:
- $\alpha$ : Es el albedo de superficie
- $\text{LAI}$ : Índice de área foliar
- $Z_{0m}$ : Momento de resistencia aerodinámica o MRA
- $\text{NDVI}$ : Índice de vegetación de diferencia normalizada y máximo $\text{NDVI}_{max}$
- $T_s$ : Temperatura de superficie y máximo valor $T_{max}$


### Flujo de calor sensible $H$

Bastiaanssen et al. (1998)

$$
H = \frac{\rho_a c_p \delta{T_a}}{r_{ah}}
$$

Dónde:
- $H$ : Flujo de calor sensible $[W/m^{2}]$
- $\rho_a$ : Densidad del aire $[kg/m^{3}]$
- $c_p$ : Calor específico del aire igual a $[1004 \space J/kg/K]$
- $\delta{T_a}$ : Gradiente de temperatura $[°C]$
- $r_{ah}$ : Resistencia aerodinámica al transporte de calor $[s/m]$ 

### $\lambda{ET}$ y $ET$ instantánea

### Evapotranspiración de cultivo $ET_c$ por día

La $ET_c$ diaria se puede calcular mediante la siguiente ecuación:

$$
ET_c[\frac{mm}{d}] = C_{rad} * \Lambda * ET_{24}
$$